In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score  #交叉验证评分
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.metrics import mean_squared_error #均方根误差
from sklearn.metrics import roc_curve, auc, recall_score, precision_score
# 导入数据，路径中要么用\\或/或者在路径前加r
dataset = pd.read_csv(r'../data/train_chuli.csv')

In [32]:
# 输出数据预览
print(dataset.head())

   PassengerId  Survived  Pclass  Sex   Age  SibSp  Parch     Fare  \
0            1         0       3    1  22.0      1      0   7.2500   
1            2         1       1    0  38.0      1      0  71.2833   
2            3         1       3    0  26.0      0      0   7.9250   
3            4         1       1    0  35.0      1      0  53.1000   
4            5         0       3    1  35.0      0      0   8.0500   

   family_size  Embarked_C  Embarked_Q  Embarked_S  
0            2           0           0           1  
1            2           1           0           0  
2            1           0           0           1  
3            2           0           0           1  
4            1           0           0           1  


In [33]:
from sklearn.model_selection import train_test_split #划分数据集
# 准备训练数据
x = dataset.drop(['Survived'], axis = 1)
y = dataset['Survived']
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=0) 
print(x_train.shape,x_test.shape,y_train.shape,y_test.shape)

(623, 11) (268, 11) (623,) (268,)


In [34]:
from matplotlib.cm import rainbow 
from sklearn.svm import SVC  
#
rf_scores = []

rf_classifier  = SVC(C=2.0,kernel = 'linear')
rf_classifier.fit(x_train, y_train)

SVC(C=2.0, kernel='linear')

SVC支持向量机-参数
核函数参数：
- C参数（惩罚参数）：控制错误分类样本对模型的惩罚力度。C越大，惩罚力度越大，模型对训练集的拟合越好，但也可能导致过拟合，default C = 1.0。
- kernel参数（核函数）：SVC中常用的核函数有线性核函数、多项式核函数、径向基函数（RBF）和sigmoid核函数。不同的核函数对应着不同的决策边界，选用合适的核函数可以提高模型的分类效果。RBF, Linear, Poly, Sigmoid
正则化参数：
- gamma参数（核函数参数）：控制模型复杂度的参数，对于RBF核函数来说，gamma越小，模型复杂度越低，对噪声数据的容忍度越高，反之gamma越大，模型复杂度越高，对噪声数据的容忍度越低。'Poly', 'RBF' and 'Sigmoid'
- degree参数（多项式核函数的次数）：多项式核函数的次数，即多项式的阶数，如果degree过大，则会导致模型过拟合。
- （5）coef0：核函数中的独立项，'RBF' and 'Poly'有效；
- （6）probablity: 可能性估计是否使用(true or false)；
- （7）shrinking：是否进行启发式；
- （8）tol（default = 1e - 3）: svm结束标准的精度;
- （9）cache_size: 制定训练所需要的内存（以MB为单位）；
- （10）class_weight: 每个类所占据的权重，不同的类设置不同的惩罚参数C, 缺省的话自适应；
- （11）verbose: 跟多线程有关，不大明白啥意思具体；
- （12）max_iter: 最大迭代次数，default = 1， if max_iter = -1, no limited;
- （13）decision_function_shape ： ‘ovo’ 一对一, ‘ovr’ 多对多 or None 无, default=None
- （14）random_state ：用于概率估计的数据重排时的伪随机数生成器的种子。

In [35]:
#评价指标
import numpy as np

# 计算混淆矩阵
def compute_confusion_matrix(precited,expected):
    part = precited ^ expected             # 对结果进行分类，亦或使得判断正确的为0,判断错误的为1
    pcount = np.bincount(part)             # 分类结果统计，pcount[0]为0的个数，pcount[1]为1的个数
    tp_list = list(precited & expected)    # 将TP的计算结果转换为list
    fp_list = list(precited & ~expected)   # 将FP的计算结果转换为list
    tp = tp_list.count(1)                  # 统计TP的个数
    fp = fp_list.count(1)                  # 统计FP的个数
    tn = pcount[0] - tp                    # 统计TN的个数
    fn = pcount[1] - fp                    # 统计FN的个数
    return tp, fp, tn, fn

# 计算常用指标
def compute_indexes(tp, fp, tn, fn):
    accuracy = (tp+tn) / (tp+tn+fp+fn)     # 准确率
    precision = tp / (tp+fp)               # 精确率
    recall = tp / (tp+fn)                  # 召回率
    F1 = (2*precision*recall) / (precision+recall)    # F1
    return accuracy, precision, recall, F1


In [36]:
rfc_score = rf_classifier.score(x_test, y_test)#测试集准确率,j即accuracy
expected = y_test.astype('int32')#预期值
# print(type(expected))
# print(expected)
rfc_pre=rf_classifier.predict(x_test)
precited = rfc_pre.astype('int32')#模型预测值
# print(type(precited))
tp, fp, tn, fn = compute_confusion_matrix(precited, expected)
accuracy, precision, recall, F1 = compute_indexes(tp, fp, tn, fn)
rfc_pre = rf_classifier.predict(x_test)
rfc_auc = metrics.roc_auc_score(y_test, rfc_pre)
rfc_rmse = np.sqrt(mean_squared_error(y_test, rfc_pre))
print('TP FP TN FN')
print('%d %d %d %d'%(tp,fp,tn,fn))
print('accuracy precision recall F1')
print('%.4lf %.4lf %.4lf %.4lf'%(accuracy, precision, recall, F1))
print('auc rmse')
print('%.4lf %.4lf'%(rfc_auc, rfc_rmse))

TP FP TN FN
71 25 143 29
accuracy precision recall F1
0.7985 0.7396 0.7100 0.7245
auc rmse
0.7806 0.4489
